In [106]:
from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv
load_dotenv()
from typing import TypedDict,Optional
from  langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from  langchain_core.prompts import PromptTemplate

In [ ]:


class Q_A(TypedDict):
    question: str
    answer: Optional[str]
    qna:Optional[str]

In [108]:
llm = HuggingFaceEndpoint(model="meta-llama/Meta-Llama-3-8B-Instruct",
                          task="text-generation",
                      )
model = ChatHuggingFace(llm=llm)

In [109]:
def give_answer(state:Q_A)->Q_A:
    prompt = PromptTemplate(template="""Answer the question.
Question: {question}
Answer:""",validate_template=True,input_variables=['question'])
    prompt = prompt.invoke({  'question':state['question']})
    result = model.invoke(prompt)
    if type(result.content) == str:
        state['answer'] = result.content
    return state
    


In [110]:
def make_qna(state:Q_A)->Q_A:
 
    prompt = PromptTemplate(template="""Can u make 5 Q and A from this : {text}
Answer:""",validate_template=True,input_variables=['text'])
    prompt = prompt.invoke({"text": state["answer"]})

    result = model.invoke(prompt)
    if type(result.content) == str:
        state['qna'] = result.content
    return state
    


In [111]:
graph = StateGraph(Q_A)


In [112]:
graph.add_node('give_answer',give_answer)
graph.add_node('make_qna',make_qna)


In [113]:
graph.add_edge(START,'give_answer')
graph.add_edge('give_answer','make_qna')
graph.add_edge('make_qna',END)




In [114]:
workflow = graph.compile()


In [115]:
result = workflow.invoke({
    'question':'What is trasnformers'
})

In [116]:
result

{'question': 'What is trasnformers',
 'answer': 'Transformers refer to a computer model known as a transformer architecture, which is a type of neural network designed to process and analyze sequential data such as language, speech, and text.\n\nThe transformer architecture was first introduced in 2017 by Google and has since become a widely used and influential model in the field of natural language processing (NLP). It is designed to handle the complexity of language by using self-attention mechanisms, which allow it to weigh the importance of different parts of the input sequence when generating the output.\n\nThe original transformer model is composed of several layers, including:\n\n1. **Encoder**: This layer takes in the input sequence and breaks it down into smaller components, such as tokens or subwords.\n2. **Decoder**: This layer generates the output sequence based on the encoded input sequence.\n3. **Self-Attention**: This mechanism allows the model to weigh the importance o